# Хакатон ЯКомпании (заключительный этап)

[Ссылка на сдачу](https://apps.skillfactory.ru/learning/course/course-v1:SkillFactory+TGUAI+SEP_2023/block-v1:SkillFactory+TGUAI+SEP_2023+type@sequential+block@ba7915b70d2143aa9546a6b2a9add87d/block-v1:SkillFactory+TGUAI+SEP_2023+type@vertical+block@b36b75cab89e4f449f296803f488a9f6)

[Ссылка на промежуточный этап](https://colab.research.google.com/drive/1RGkRGdgyEG2IysAYeA-P2fpkPlPwxZj3?usp=sharing) (Анализ, гипотезы, пробные подходы)

Команда 3:

Рогов И. rogovins@mail.ru

Смирнов Ю. h4rdwork@yandex.ru

Стрыгин Д. d.strygin@mail.ru

In [ ]:
!pip install pymongo pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 18.5 MB/s eta 0:00:00


In [ ]:
!pip install sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 39.8 MB/s eta 0:00:00


In [ ]:
from pymongo import MongoClient
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
import faiss
import pickle

# Параметры подключения
uri = "mongodb://mongodb01.kube01.icncd.ru:27017/imarketplacetesting?authSource=$external&authMechanism=MONGODB-X509"

# Путь к файлу сертификата клиента (ключевой файл)
client_cert_file = '/content/tguai.pem'  # Загруженный в локальный корень файл

# Путь к файлу корневого сертификата
ca_file = '/content/root_ca.crt'  # Загруженный в локальный корень файл

# Настройка параметров SSL для подключения
client = MongoClient(uri,
                     tls=True,
                     tlsCertificateKeyFile=client_cert_file,
                     tlsCAFile=ca_file)

# Получение базы данных и коллекции
db = client['imarketplacetesting']
collection = db['parsedspecialists']

# Загрузка модели
model = SentenceTransformer('distiluse-base-multilingual-cased-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.69k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

Нижележащий код на показе можно пропустить

он векторизует элементы резюме кандидатов напрямую из МангоБД. Чтобы не выполнять эту долгую работу каждый раз, мы сохраняем результат в Пикл-файл. И в дальнейшем (на показе), просто его грузим с готовыми векторами резюме (разумеется, при существенном обновлении БД, надо будет делать новую векторизацию, или векторизацию по добавлению новой инфы, но для демонстрации сути подхода, достаточно сделать так)

In [ ]:
def extract_resume_data(limit=0):
    # Получить документы из коллекции
    if limit > 0:
        cursor = collection.find().limit(limit)
    else:
        cursor = collection.find()

    resumes = []
    for document in cursor:
        # Accessing the nested 'data' field
        data = document.get('data', {})

        # Извлечение ID
        resume_id = str(document.get('_id'))

        # Извлечение нужных полей
        skills = data.get('skills', '')
        # Если 'experience' вложено в 'data'
        descriptions = [exp.get('description', '') for exp in data.get('experience', []) if 'description' in exp]

        # Печать для отладки
        # print(f"ID: {resume_id}")
        # print(f"Skills: {skills}")
        # print(f"Descriptions: {descriptions}")

        # Сохранение ID и данных
        resumes.append({'id': resume_id, 'skills': skills, 'descriptions': descriptions})

    return resumes

# Извлечение данных резюме
resumes_data = extract_resume_data(limit=100) # Здесь можно задать лимит (при отладке брал 3, в боевой версии 0)

# Создание векторных представлений
def clean_text(text):
    """Возвращает пустую строку, если входной `text` равен `None`."""
    return text if text is not None else ""

def create_embeddings_for_resumes(resumes):
    resume_embeddings = []
    for resume in resumes:
        # Очистка данных
        skills = clean_text(resume.get('skills', ''))
        descriptions = [clean_text(desc) for desc in resume.get('descriptions', [])]

        # Векторизация
        skills_embedding = model.encode(skills, convert_to_tensor=True).numpy()
        descriptions_embeddings = [model.encode(desc, convert_to_tensor=True).numpy() for desc in descriptions]

        resume_embeddings.append({
            'id': resume['id'],
            'skills': skills_embedding,
            'descriptions': descriptions_embeddings
        })

    return resume_embeddings

resume_embeddings = create_embeddings_for_resumes(resumes_data)



In [ ]:
# Сохранение векторных представлений в файл
def save_embeddings_to_file(embeddings, filename='resume_embeddings.pkl'):
    with open(filename, 'wb') as file:
        pickle.dump(embeddings, file)

# Загрузка векторных представлений из файла
def load_embeddings_from_file(filename='resume_embeddings.pkl'):
    with open(filename, 'rb') as file:
        return pickle.load(file)

# Сохранение в файл
save_embeddings_to_file(resume_embeddings)

# Пример загрузки из файла
#loaded_resume_embeddings = load_embeddings_from_file()

# Основная часть для поиска кандидата.

1) Ввод айди вакансии

In [ ]:
import json
from bson import json_util


In [ ]:
# Получение базы данных и коллекции
db = client['imarketplacetesting']
resumes_collection = db['parsedspecialists']
jobs_collection = db['jobs']

def get_job_requirements(job_id_or_index):
    # Если ввод — это число, считаем это индексом в коллекции
    if isinstance(job_id_or_index, int):
        # Получаем документ по индексу
        job = list(jobs_collection.find().skip(job_id_or_index).limit(1))
        if not job:
            raise ValueError("Вакансия с данным индексом не найдена.")
        job_doc = job[0]
    else:
        # Считаем, что это ID вакансии
        job_doc = jobs_collection.find_one({"_id": job_id_or_index})
        if not job_doc:
            raise ValueError("Вакансия с данным ID не найдена.")

    # Извлечение требований
    requirements = job_doc.get("requirements", [])
    ru_requirements = [req["requirement"]["ru"] for req in requirements]

    # Печать требований
    for idx, req in enumerate(ru_requirements):
        print(f"Требование {idx + 1}: {req}")

    return requirements

# Используем ввод для получения требований
job_id = input("Введите ID или порядковый номер вакансии: ")
requirements = get_job_requirements(job_id)

Введите ID или порядковый номер вакансии: 2


ValueError: Вакансия с данным ID не найдена.

2. Векторизация требований и проверка кандидатуры

In [ ]:
def vectorize_requirements(requirements):
    # Обработка требований: текст и mandatory
    texts = [req["requirement"]["ru"] for req in requirements]
    mandatory_flags = [req.get("mandatory", False) for req in requirements]

    # Векторизация текста
    embeddings = model.encode(texts)
    return embeddings, mandatory_flags

requirement_embeddings, mandatory_flags = vectorize_requirements(requirements)

3. Загрузка и проверка кандидатов, вычисление соответствия

In [ ]:
def load_embeddings_from_file(filename='resume_embeddings.pkl'):
    with open(filename, 'rb') as file:
        return pickle.load(file)

def calculate_match(candidate_embeddings, requirement_embeddings, mandatory_flags):
    total_score = 0
    for idx, req_emb in enumerate(requirement_embeddings):
        scores, _ = candidate_embeddings.search(np.array([req_emb]), k=1)
        weight = 1 if mandatory_flags[idx] else 0.3
        total_score += scores[0][0] * weight
    avg_score = total_score / len(requirement_embeddings)
    return avg_score

def find_best_candidates(requirement_embeddings, mandatory_flags, top_k=5):
    resume_embeddings = load_embeddings_from_file()
    best_candidates = []
    index = faiss.IndexFlatL2(requirement_embeddings.shape[1])

    for candidate in resume_embeddings:
        skills_embedding = faiss.index_cpu_to_all_gpus(candidate['skills'])
        descriptions_embeddings = [faiss.index_cpu_to_all_gpus(desc) for desc in candidate['descriptions']]
        match_score = calculate_match(descriptions_embeddings, requirement_embeddings, mandatory_flags)
        best_candidates.append((candidate['id'], match_score))

    # Сортировка кандидатов по соответствию
    best_candidates.sort(key=lambda x: x[1], reverse=True)
    return best_candidates[:top_k]

best_candidates = find_best_candidates(requirement_embeddings, mandatory_flags)

for idx, (candidate_id, score) in enumerate(best_candidates):
    print(f"\nКандидат {idx + 1} (ID: {candidate_id}):")
    resume = resumes_collection.find_one({"_id": candidate_id})
    print(f"Скилы: {resume['skills']}")
    print(f"Навыки: {resume['experience']}")